<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# XGBoost

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Right now I am getting a useless warning every time I fit an `XGBoost` model.
# This line of code prevents warnings from being displayed. Not generally
# recommended.
warnings.filterwarnings(action='ignore')

In [ ]:
%matplotlib inline

"Gradient boosting," like bagging, is a general method for training decision tree ensembles.

XGBoost ("eXtreme Gradient Boosting") is a particular implementation of gradient boosted decision trees. It is popular on Kaggle because it is both fast to train and often gives excellent predictive performance.

## Getting Started with XGBoost

We will use the `xgboost` library instead of scikit-learn for this lesson. Scikit-learn has [`GradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) and [`GradientBoostingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) classes, but they lack some of the tricks that have made the `xgboost` library popular.

`xgboost` provides estimators that use the same interface as scikit-learn's, so we will not need to change our approach.

In [ ]:
# Install the xgboost library
# !pip install xgboost

In [ ]:
# Import the xgboost package
# /scrub/
import xgboost as xgb

In [ ]:
# Instantiate an XGBoost regressor
# /scrub/
xgb_reg = xgb.XGBRegressor()

**Exercise (20 mins., in pairs)**

- Load the Ames housing dataset from `assets/data/ames_train.csv` in this lesson's base directory.

In [ ]:
# /scrub/
ames_df = pd.read_csv('../assets/data/ames_train.csv')

- Create a feature matrix DataFrame `X` containing all of the numeric columns from the Ames dataset except "Id" and the target column "SalePrice". Drop "OverallQual" to make things more interesting -- that very is very predictive but expensive to collect.

In [ ]:
# /scrub/
target_col = 'SalePrice'

X = (ames_df
     .select_dtypes(['int64', 'float64'])
     .drop(['Id', 'OverallQual', target_col], axis='columns')
    )

- Create a target vector Series `y` with the values of the variable "SalePrice".

In [ ]:
# /scrub/
y = ames_df.loc[:, target_col]

- Do a simple train/test split on `X` and `y`.

In [ ]:
# /scrub/
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

- Fit an `XGBRegressor` on the training data.

In [ ]:
# /scrub/
xgb_reg.fit(X_train, y_train)

- Get an R^2 score for the training set.

In [ ]:
# /scrub/
xgb_reg.score(X_train, y_train)

- Get an R^2 score for the test set.

In [ ]:
# /scrub/
xgb_reg.score(X_test, y_test)

- Is your model overfitting, underfitting, both, or neither? How do you know?

/scrub/

Overfitting somewhat. The score on the training set is close to 1, so it does not seem to be underfitting, and the gap between training and test scores is moderately large.

- Load the Titanic dataset (located in this lesson's  `assets/data` directory).

In [ ]:
# /scrub/
titanic = pd.read_csv('../assets/data/titanic.csv')

- Create a feature matrix `X` by dropping "PassengerId", "Name", "Ticket", and the target column "Survived." Dummy-code string columns as needed. Do NOT drop or impute missing values -- XGBoost handles them internally!

![](https://media.giphy.com/media/12NUbkX6p4xOO4/giphy.gif)

In [ ]:
# /scrub/
X = titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis='columns')

In [ ]:
# /scrub/
X = pd.get_dummies(X, columns=['Sex', 'Embarked'])

- Create a target Series `y` with the values of "Survived."

In [ ]:
# /scrub/
y = titanic.loc[:, 'Survived']

- Instantiate an `XGBClassifier`.

In [ ]:
# /scrub/
xgb_clas = xgb.XGBClassifier()

- Fit and score the classifier on the entire dataset.

In [ ]:
# /scrub/
xgb_clas.fit(X, y)
xgb_clas.score(X, y)

- Get your classifier's score on held-out data using ten-fold cross-validation on the Titanic dataset, shuffling the rows before taking the folds.

In [ ]:
# /scrub/
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
# /scrub/
kf = KFold(10, shuffle=True)

In [ ]:
# /scrub/
scores = cross_val_score(xgb_clas, X, y, cv=kf)

In [ ]:
# /scrub/
np.array(scores).mean()

- Is your model overfitting, underfitting, both, or neither? How do you know?

/scrub/

Maybe neither, or slightly overfitting. The score on the training set seems pretty good, so it does not seem to be underfitting, and the gap between training and test scores is not very big.

$\blacksquare$

## XGBoost vs. Random Forests

### Similarities

Random forests and XGBoost both produce tree ensembles, and the provide many of the same parameters to reduce overfitting.

### Differences

#### Gradient Boosting vs. Bagging

- Bagging involves training each tree *independently* on a different *bootstrap sample*.
- Gradient boosting involves training each tree *sequentially to reduce the residual errors left by its predecessors*.

See [the official `xgboost` library documentation](https://xgboost.readthedocs.io/en/latest/tutorials/model.html) and Chapter 10 of [Elements of Statistical Learning](https://web.stanford.edu/~hastie/Papers/ESLII.pdf) for details.

#### Handling Missing Values

At each split for a given variable, XGBoost simply learns whether sending items with missing values left or right gives better results. This approach has a few advantages:

- It is automatic.
- Unlike dropping rows or columns, it allows you to use all of the values you do have.
- Unlike imputation, it treats "missing" as its own value rather than replacing it with some other value that might be wrong.

## Tuning XGBoost

`XGBoost` provides many options that you can tune to improve predictive performance.

### `n_estimators` and `learning_rate`

The `learning_rate` controls how "aggressive" each tree is in trying to correct the errors of its predecessors.

- If it is too low, then getting good predictive performance will require a large value for `n_estimators` (and thus a lot of time).
- If it is too high, then the algorithm will keep overshooting the target and won't coverge to good results.

Unlike with a random forest, setting `n_estimators` too high can hurt predictive performance with boosting because it leads to overfitting.

### Addressing Overfitting

#### Reducing Model Complexity

One way to address overfitting is to restrict model complexity more or less directly. `xgboost` provides many options for this purpose:

- Restricting tree shape
    - `max_depth` / `max_leaf_nodes` puts a hard limit on the depth or number of leaves in each tree
    - `gamma` is the minimum loss reduction required in order to make another split
    - `min_child_weight` is the minimum number of observations required in each child node in order to make a split, adjusted for the weight that is placed on each observation
- Restricting sizes of weights: `reg_lambda` and `reg_alpha` provide L1 and L2 regularization on sample weights, respectively

#### Adding Randomness

Another way to address overfitting when ensembling is to add randomness to the process of training each item in the ensemble.

- `subsample` specifies what proportion of the data is used to train each tree.
- `colsample_bytree` and `colsample_bylevel` specify what proportion of the features are available at the tree and split level, respectively.

### Example

We will use this general approach to tune our model:

- Find the optimal number of trees with default learning rate.
- Tune additional parameters.
- Lower learning rate and increase the number of trees.

#### Find Optimal Number of Trees with Default Learning Rate

In [ ]:
# Split data by column
# /scrub/
target_col = 'SalePrice'

X = ames_df.select_dtypes(['int64', 'float64']).drop(['Id', 'OverallQual', target_col], axis='columns')
y = ames_df.loc[:, target_col]

In [ ]:
# Instantiate model
# /scrub/
xgb_reg = xgb.XGBRegressor(learning_rate=0.1, max_depth=1)

In [ ]:
# Fit and score on all data
# /scrub/
xgb_reg.fit(X, y)
print(xgb_reg.score(X, y))

In [ ]:
# Score with 5-fold CV
# /scrub/
kf = KFold(n_splits=5, shuffle=True, random_state=1)
np.array(cross_val_score(xgb_reg, X, y, cv=kf)).mean()

In [ ]:
# Vary number of trees
# /scrub/
for num_trees in range(100, 1000, 100):
    xgb_reg = xgb.XGBRegressor(n_estimators=num_trees, max_depth=1)
    print(num_trees, np.array(cross_val_score(xgb_reg, X, y, cv=kf)).mean())

In [ ]:
# Fix number of trees
# /scrub/
xgb_reg = xgb.XGBRegressor(n_estimators=400)

#### Tune Additional Parameters

scikit-learn has a `GridSearchCV` class that will run a model with various hyperparameter combinations and identify the combination that generated the best cross-validation scores.

In [ ]:
# Try a few values for "max_depth" and "min_child_weight"
# /scrub/
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [1, 4, 7], 'min_child_weight': [1, 10, 100]
}
grid_search = GridSearchCV(
    xgb_reg,
    param_grid=param_grid,
    cv=kf
)
grid_search.fit(X_train, y_train)

In [ ]:
# Find out best parameters and their score
# /scrub/
grid_search.best_params_, grid_search.best_score_

In [ ]:
# Try a few values for "subsample"
# /scrub/
param_grid = {
    'subsample': np.linspace(0.5, 1, 4)
}
xgb_reg = xgb.XGBRegressor(n_estimators=500, max_depth=4, min_child_weight=1)
grid_search = GridSearchCV(
    xgb_reg, 
    param_grid=param_grid,
    cv=4
)
grid_search.fit(X_train, y_train)

In [ ]:
# Find out best parameters and their score
# /scrub/
grid_search.best_params_, grid_search.best_score_

In [ ]:
# Get report on grid search results
# /scrub/
pd.DataFrame(grid_search.cv_results_)

The effect of one hyperparameter typically depends on the values of other hyperparameters -- for instance, increasing "max_depth" will have no effect if "min_child_weight" is sufficiently large. For this reason, it is generally valuable to do grid searches over multiple parameters simultaneously, rather than fixing one hyperparameter at a time. However, testing many combinations of many parameters can take a long time.

#### Lower Learning Rate and Increase the Number of Trees.

In [ ]:
# Divide the learning rate by 10 and vary number of trees
# /scrub/
for num_trees in range(100, 3101, 500):
    xgb_reg = xgb.XGBRegressor(
        learning_rate=.01,
        n_estimators=num_trees,
        max_depth=4,
        min_child_weight=1,
        subsample=0.67,
    )
    print(num_trees, np.array(cross_val_score(xgb_reg, X, y, cv=kf)).mean())

**Exercise (open-ended, in pairs)**

- Create the best XGBoost model you can for the Titanic dataset, as measured by accuracy in five-fold cross-validation. Use `GridSearchCV` at least once to search over at least two hyperparameters at a time.

In [ ]:
titanic = pd.read_csv('../assets/data/titanic.csv')
X = titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis='columns')
X = pd.get_dummies(X, columns=['Sex', 'Embarked'])
y = titanic.loc[:, 'Survived']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
xgb_clas = xgb.XGBClassifier()
xgb_clas.fit(X, y)
xgb_clas.score(X, y)

In [ ]:
param_grid = {
    'max_depth': [1, 4, 7], 'min_child_weight': [1, 10, 100]
}
grid_search = GridSearchCV(
    xgb_clas,
    param_grid=param_grid,
    cv=kf
)
grid_search.fit(X_train, y_train)

In [82]:
# Find out best parameters and their score
# /scrub/
grid_search.best_params_, grid_search.best_score_

({'max_depth': 4, 'min_child_weight': 1}, 0.8218562874251497)

In [83]:
# Get report on grid search results
# /scrub/
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_child_weight,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.162820,0.026252,0.009788,0.007628,1,1,"{'max_depth': 1, 'min_child_weight': 1}",0.776119,0.783582,0.820896,...,0.806886,0.026514,4,0.825843,0.812734,0.797753,0.805607,0.805607,0.809509,0.009443
1,0.118605,0.005466,0.011210,0.008202,1,10,"{'max_depth': 1, 'min_child_weight': 10}",0.753731,0.768657,0.820896,...,0.794910,0.035261,6,0.807116,0.814607,0.797753,0.798131,0.796262,0.802774,0.007043
2,0.153805,0.080595,0.007999,0.003953,1,100,"{'max_depth': 1, 'min_child_weight': 100}",0.559701,0.619403,0.611940,...,0.613772,0.030642,7,0.627341,0.612360,0.614232,0.611215,0.603738,0.613777,0.007662
3,0.200006,0.022901,0.007805,0.003762,4,1,"{'max_depth': 4, 'min_child_weight': 1}",0.768657,0.783582,0.828358,...,0.821856,0.040277,1,0.917603,0.910112,0.906367,0.904673,0.908411,0.909433,0.004479
4,0.188010,0.035018,0.008404,0.003265,4,10,"{'max_depth': 4, 'min_child_weight': 10}",0.768657,0.761194,0.828358,...,0.811377,0.039168,3,0.859551,0.874532,0.870787,0.850467,0.848598,0.860787,0.010445
5,0.151013,0.019148,0.006999,0.002097,4,100,"{'max_depth': 4, 'min_child_weight': 100}",0.559701,0.619403,0.611940,...,0.613772,0.030642,7,0.627341,0.612360,0.614232,0.611215,0.603738,0.613777,0.007662
6,0.364820,0.073922,0.016004,0.014655,7,1,"{'max_depth': 7, 'min_child_weight': 1}",0.776119,0.783582,0.798507,...,0.806886,0.027526,4,0.960674,0.964419,0.943820,0.951402,0.966355,0.957334,0.008492
7,0.208009,0.015158,0.007604,0.001850,7,10,"{'max_depth': 7, 'min_child_weight': 10}",0.776119,0.768657,0.828358,...,0.814371,0.035628,2,0.857678,0.874532,0.867041,0.848598,0.854206,0.860411,0.009262
8,0.108804,0.004117,0.005002,0.000892,7,100,"{'max_depth': 7, 'min_child_weight': 100}",0.559701,0.619403,0.611940,...,0.613772,0.030642,7,0.627341,0.612360,0.614232,0.611215,0.603738,0.613777,0.007662


In [86]:
xgb_clas = xgb.XGBClassifier(max_depth= 4, min_child_weight= 1)
xgb_clas.fit(X, y)
xgb_clas.score(X, y)

0.9001122334455668

In [88]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(5, shuffle=True)
scores = cross_val_score(xgb_clas, X, y, cv=kf)
np.array(scores).mean()

0.828253091456908

$\blacksquare$

## Summary

- `XGBoost` is a popular decision tree ensemble algorithm.
- `XGBoost` uses gradient boosting, meaning that each tree attempts to correct the errors of previous trees.
- scikit-learn's `GridSearchCV` helps with testing hyperparameter values.